<a href="https://colab.research.google.com/github/ranitha84/gesture_recognition/blob/main/Neural_Nets_Project_Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import os
import datetime
import os
import cv2


We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
from keras import backend as K
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers


rn.seed(30)
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
drive_path = '/content/drive/MyDrive'

train_doc = np.random.permutation(open(drive_path+'/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
val_doc = np.random.permutation(open(drive_path+'/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
batch_size = 32

image_width = 120
image_height = 120

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
class CustomDataGenerator:

  def __init__(self, width=120, height=120, frames=30, channels=3, 
               crop = True, normalize = True, resize=False ):
    self.width= width
    self.height = height
    self.frames = 30
    self.channels = channels
    self.crop = crop
    self.normalize= normalize
    self.resize= resize

  def __crop_image(self, image):
    if image.shape[0] != image.shape[1]:
      return image[0:120, 20:140]
    else:
      return image    

  def __resize_image(self,image):
    return cv2.resize(image, (self.width,self.height), interpolation = cv2.INTER_AREA)

  def __normalize_image(self,image):
    return image/255.0

  def __initialize_batch_data(self,batch_size):
     batch_data = np.zeros((batch_size,self.frames,self.width,self.height,self.channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
     batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
     return batch_data, batch_labels


  def generator(self, source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0, self.frames)] 
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list)/batch_size)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data, batch_labels = self.__initialize_batch_data(batch_size)
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item],cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    # crop image
                    if self.crop:
                      image = self.__crop_image(image)
                    # resize the image
                    if self.normalize:
                      image=  self.__normalize_image(self.__resize_image(image))
                    else:
                       image = self.resize_image(image)
                    batch_data[folder,idx] = image
                    #batch_data[folder,idx,:,:,0] = normalize_image(image)
                    #batch_data[folder,idx,:,:,1] = normalize_image(image)
                    #batch_data[folder,idx,:,:,2] = normalize_image(image)
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
                batch_size = len(folder_list) - (batch_size*num_batches)
                yield self.generator(source_path, folder_list, batch_size) 

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:

train_path = drive_path+'/notebooks/storage/Final_data/Collated_training/train'
val_path = drive_path+'/notebooks/storage/Final_data/Collated_training/val'


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
class ModelGenerator:

  def __init__(self, input_shape = (30,120,120, 3),num_classes = 5):
    self.input_shape= input_shape
    self.num_classes = num_classes

  
  def model_cv3D(self):
      model = Sequential()
      model.add(Conv3D(
          8, (3,3,3), activation='relu', input_shape=self.input_shape
      ))
      model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
      model.add(Conv3D(16, (3,3,3), activation='relu'))
      model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
      model.add(Conv3D(32, (3,3,3), activation='relu'))
      model.add(Conv3D(32, (3,3,3), activation='relu'))
      model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
      model.add(Conv3D(64, (2,2,2), activation='relu'))
      model.add(Conv3D(64, (2,2,2), activation='relu'))
      model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

      model.add(Flatten())
      model.add(Dense(512))
      model.add(Dropout(0.5))
      model.add(Dense(256))
      model.add(Dropout(0.5))
      model.add(Dense(self.num_classes, activation='softmax'))

      return model



# Train Model

- This method takes the model, batch size and the number of epochs to train the model

- Its the takes the train and validation genarators to the train the data on the data set

In [8]:
def train_model( model, batch_size, num_epochs, train_generator, val_generator, optimizer=None):
    curr_dt_time = datetime.datetime.now()
    num_train_sequences = len(train_doc)
    print('# training sequences =', num_train_sequences)
    num_val_sequences = len(val_doc)
    print('# validation sequences =', num_val_sequences)
    print ('# epochs =', num_epochs)

    print('Number of epochs {}'.format(num_epochs))
    print('Batch Size {}'.format(batch_size))
    print('Current Date time {}'.format(curr_dt_time))

    if optimizer == None:
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print (model.summary())


    model_name = drive_path+'/'+'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    if not os.path.exists(model_name):
        os.mkdir(model_name)
            
    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, 
                                 monitor='val_loss', 
                                 verbose=1, 
                                 save_best_only=True, 
                                 save_weights_only=False, 
                                 mode='auto', 
                                 save_freq=5*batch_size)

    LR = ReduceLROnPlateau(monitor='val_loss',factor=0.2, patience=2, model='auto', verbose=1)

    early_stopping = EarlyStopping(monitor='val_loss',patience=5, mode='auto', verbose=1),

    callbacks_list = [checkpoint, LR,early_stopping]

    if (num_train_sequences%batch_size) == 0:
      steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1

    model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
data_generator = CustomDataGenerator()

train_generator = data_generator.generator(train_path, train_doc, batch_size)
val_generator = data_generator.generator(val_path, val_doc, batch_size)

model_generator = ModelGenerator()

model = model_generator.model_cv3D()

num_epochs =20

train_model(model,batch_size,num_epochs,train_generator,val_generator)

# training sequences = 663
# validation sequences = 100
# epochs = 20
Number of epochs 20
Batch Size 32
Current Date time 2021-02-25 03:00:43.215446
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 22,